# 会話型インターフェイス - AI21 LLM 搭載チャットボット

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

このノートブックでは、Amazon Bedrock の基本モデル (FM) を使用してチャットボットを構築します。このユースケースでは、チャットボットを構築するための FM として Jurassic を使用しています。

## 概要

チャットボットやバーチャルアシスタントなどの会話型インターフェースを使用して、顧客のユーザーエクスペリエンスを向上させることができます。チャットボットは、自然言語処理 (NLP) と機械学習アルゴリズムを使用してユーザーのクエリを理解し、それに応答します。チャットボットは、顧客サービス、営業、電子商取引などのさまざまなアプリケーションで使用可能で、ユーザーに迅速かつ効率的に対応することができます。ウェブサイト、ソーシャルメディアプラットフォーム、メッセージングアプリなど、さまざまなチャネルからアクセスできます。

## Amazon Bedrock を利用したチャットボット

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## ユースケース
1. **チャットボット(基本)** -  FM モデルの Zero-Shot チャットボット
2. **プロンプトテンプレートを使用したチャットボット** - プロンプトテンプレートにコンテキストが提供されているチャットボット
3. **ペルソナチャットボット** - 役割が定義されたチャットボット。例：キャリアコーチとヒューマンインタラクション
4. **コンテキストを意識したチャットボット** - 埋め込み生成による外部ファイルを利用したコンテキストを意識したチャットボット

## Amazon Bedrock でチャットボットを構築するための Langchain フレームワーク
チャットボットのような会話型インターフェースでは、短期的にも長期的にも、以前のやりとりを覚えておくことが非常に重要です。

LangChain は 2 つの形式のメモリコンポーネントを提供します。まず、LangChain は以前のチャットメッセージを管理および操作するためのヘルパーユーティリティを提供します。これらはモジュラーで、使用方法に関係なく役立つように設計されています。次に、LangChain はこれらのユーティリティをチェーンに組み込む簡単な方法を提供します。
これにより、さまざまなタイプの抽象化を簡単に定義して操作できるようになるため、強力なチャットボットの構築が容易になります。

## コンテキストを利用したチャットボットの構築-主な要素
コンテキストを意識したチャットボットを構築する最初のプロセスは、コンテキストの**埋め込みを生成**することです。通常、埋め込みモデルを実行して埋め込みを生成し、それをある種のベクトルストアに保存する取り込みプロセスがあります。この例では、GPT-J 埋め込みモデルを使用しています。

![Embeddings](./images/embeddings_lang.png)

2 番目のプロセスは、ユーザーリクエストのオーケストレーション、インタラクション、結果の呼び出しと出力です。

![Chatbot](./images/chatbot_lang.png)

## アーキテクチャ [コンテキストを意識したチャットボット]
![4](./images/context-aware-chatbot.png)

## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrock に接続する必要があります。

セットアップの仕組みと ⚠️ **whether you might need to make any changes** についての詳細は、[Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ja.ipynb) を参照してください。

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

この Notebook では、追加の依存関係が必要になります：

- [FAISS](https://github.com/facebookresearch/faiss)...　埋め込みベクトルを保存するため
- [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/)... Notebook のインタラクティブな UI ウィジェットのため
- [PyPDF](https://pypi.org/project/pypdf/)... PDF ファイルを操作するため

In [ ]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.249 "pypdf>=3.8,<4"

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## チャットボット (基本 - コンテキストなし)

#### LangChain の CoversationChain を使って会話を始める
チャットボットは以前のやりとりを覚えておく必要があります。会話型メモリによってそれが可能になります。会話型メモリを実装する方法はいくつかあります。LangChain のコンテキストでは、これらはすべて CoversationChain の上に構築されています。

注意 : モデルの出力は非決定的です

In [ ]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory

ai21_llm = Bedrock(model_id="ai21.j2-jumbo-instruct", client=boto3_bedrock)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=ai21_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="Hi there!"))

#### 新しい質問

モデルが最初のメッセージに応答しました。いくつか質問してみましょう

In [ ]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

#### 質問を構築する

garden という言葉は用いずに質問をして、モデルが以前の会話を理解できるか確認してみましょう

In [ ]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

#### 会話を終了する

In [ ]:
print_ww(conversation.predict(input="That's all, thank you!"))

## プロンプトテンプレートを使用したチャットボット(Langchain)

PromptTemplate は、この入力の構築を担当します。LangChain には、プロンプトの作成と操作を簡単にするためのクラスと関数がいくつか用意されています。ここでは、こちらのデフォルトのプロンプトテンプレートを使用します。 [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate

chat_history = []

# turn verbose to true to see the full logs and documents
qa= ConversationChain(
    llm=ai21_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

print(f"ChatBot:DEFAULT:PROMPT:TEMPLATE: is ={qa.prompt.template}")

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

それではチャットをしてみましょう。

In [ ]:
chat = ChatUX(qa)
chat.start_chat()

## ペルソナチャットボット

AI アシスタントはキャリアコーチの役割を果たします。ロールプレイダイアログでは、チャットを開始する前にユーザーメッセージを設定する必要があります。会話バッファメモリはダイアログの事前入力に使用されます。

In [ ]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Context:You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am career coach and give career advice")
ai21_llm = Bedrock(model_id="ai21.j2-jumbo-instruct",client=boto3_bedrock)
conversation = ConversationChain(
     llm=ai21_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="What are the career options in AI?"))

##### このペルソナの専門ではない質問をしてみましょう。モデルがその質問に答えたり、その理由を述べたりしてはいけません。

In [ ]:
conversation.verbose = False
print_ww(conversation.predict(input="How to fix my car?"))

## コンテキストを意識したチャットボット
このユースケースでは、チャットボットが渡されたコンテキストをもとに質問に回答するようにします。csv ファイルを取得し、Titan 埋め込みモデルを使用してベクトルを作成します。このベクトルは FAISS に保存されます。チャットボットに質問があった場合は、このベクトルを提供して回答を取得します。 

#### Titan 埋め込みモデルを使う - 文書の埋め込みを生成できるようにする

埋め込みは、単語、フレーズ、またはその他の個別の項目を連続ベクトル空間のベクトルとして表現する方法です。これにより、機械学習モデルはこれらの表現に対して数学的な操作を実行し、それらの間の意味的な関係を捉えることができます。


こちらは RAG に使用されます。 [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/) 

その他の可能な埋め込みはこちらです。[LangChain Embeddings](https://python.langchain.com/en/latest/reference/modules/embeddings.html)

In [ ]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain import PromptTemplate

br_embeddings = BedrockEmbeddings(client=boto3_bedrock)

#### ドキュメント検索のための埋め込みを作成する

#### ベクトルストアインデクサ

こちらが埋め込みデータを保存し、マッチさせるものです。このノートブックには Chroma と FAISS が搭載されており、一時的にメモリに保存されます。ベクトルストア API は [こちら](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html) で入手できます。

SageMaker Embeddings の独自のカスタム実装を使用します。この実装では、埋め込みを行うモデルを呼び出すために SageMaker エンドポイントへの参照が必要です。これは FAISS や Chroma がメモリに保存するのに使用され、ユーザーがクエリを実行するたびに使われます。

#### ベクトルストアとしての FAISS

メモリベクトルストアの [FAISS](https://arxiv.org/pdf/1702.08734.pdf) については、こちらをご覧ください。

Chroma

[Chroma](https://www.trychroma.com/) はとてもシンプルなベクトル検索データベースです。core-API はわずか 4 つの関数で構成されているため、ユーザーはメモリ内にドキュメントベクトルストアを構築できます。デフォルトでは、Chroma は Hugging Face Transformers ライブラリを使用してドキュメントをベクトル化します。

Weaviate

[Weaviate](https://github.com/weaviate/weaviate) はとてもおしゃれなツールです。Weaviate はベクトル検索をサポートする GraphQL API を提供しているだけではありません。ユーザーは Weaviate の組み込みモジュールまたはカスタムモジュールを使用してコンテンツをベクトル化することができます。

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

s3_path = f"s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")

vectorstore_faiss_aws = FAISS.from_documents(
    documents=docs,
    embedding = br_embeddings, 
    #**k_args
)

print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")


#### 簡単なローコードテストを実行する 

LangChain が提供する Wrapper クラスを使用して、ベクターデータベースストアをクエリし、関連するドキュメントを得ることができます。裏側は、すべてデフォルト値で QA チェーンを実行するのみです。

In [ ]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=ai21_llm))

#### チャットボットアプリケーション

チャットボットには、コンテキスト管理、履歴、ベクトルストア、その他多くのものが必要です。まず、ConversationalRetrievalChain から始めます。

これは会話メモリと RetrievalQAChain を使用し、フォローアップの質問に使用できるチャット履歴を提供することができます。

出典:https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

verbose を True に設定すると、裏側で何が起こっているかをすべて確認することができます。

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT


def create_prompt_template():
    _template = """{chat_history}

Answer only with the new question.
How would you ask the question considering the previous conversation: {question}
Question:"""
    CONVO_QUESTION_PROMPT = PromptTemplate.from_template(_template)
    return CONVO_QUESTION_PROMPT

memory_chain = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
chat_history=[]

#### ConversationRetrievalChain に使用されるパラメータ

Retriever: ベクトルストアをバックエンドとして使用した VectorRestoreRetriver を使用しました。テキストを取得するには、search_type として similarity または mmr の 2 つの検索タイプを選択できます。similarity は、Retriever オブジェクト内の類似検索を使用して、質問ベクトルに最も近いテキストチャンクベクトルを選択します。

memory: 履歴を保存するためのメモリストアです。

condense_question_prompt: ユーザーからの質問があった際、以前の会話とユーザーの質問を使用して、独立した質問を作成します。

chain_type: チャット履歴が長く、コンテキストに合わない場合は、このパラメータを使用します。オプションは stuff、refine、map_reduce、map-rerank です。

注意:質問が提供されたコンテキストの範囲外である場合、モデルはわからないと答えます

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=ai21_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    #verbose=True,
    #condense_question_prompt=CONDENSE_QUESTION_PROMPT, # create_prompt_template(), 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=100
)

qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}:

Use at maximum 3 sentences to answer the question. 

{question}:

If the answer is not in the context say "Sorry, I don't know, as the answer was not found in the context."

Answer:""")

チャットを開始しましょう！

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

### 今回のデモでは、 AI21 Labs Jurassic LLM を使用して以下のパターンで会話型インターフェイスを作成しました

1. チャットボット (基本 - コンテキストなし)

2. プロンプトテンプレートを使用したチャットボット (Langchain)

3. ペルソナチャットボット

4. コンテキストを意識したチャットボット